In [4]:
import os,sys,math,numpy
import PIL 
import matplotlib.pyplot as plt
from urllib.request import urlopen

# you must have openvisus installed
# python -m pip install --upgrade OpenVisus
from OpenVisus import *

# LoadImage (example: img=LoadImage("https://github.com/sci-visus/OpenVisus/raw/master/datasets/cat/gray.png"))
def LoadImage(filename):
	return numpy.asarray(PIL.Image.open(urlopen(filename)  if filename.startswith('http') else filename))

def ShowImage(data,width=10):
	ratio=float(data.shape[1])/data.shape[0]
	fig = plt.figure(figsize = (width,width*ratio))
	ax = fig.add_subplot(1,1,1)
	ax.imshow(data, origin='lower')
	plt.show()

In [2]:
db=LoadDataset('http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1')
dims=db.getLogicBox()[1]
print("DIMS",dims)

fields=[db.getField(name) for name in db.getFields()]
for field in fields:
    dtype=field.dtype
    print("Field",field.name, dtype.toString(),
         "{}GB".format(int(dims[0]*dims[1]*dims[2]*dtype.getByteSize()/(1024*1024*1024))))

DIMS [2048, 2048, 2048]
Field DATA uint8 8GB


In [11]:
import ipywidgets

quality=-6

def ExtractData(x,y,z,num_refinements,):
    for data in db.read(x=x,y=y,z=z,num_refinements=num_refinements,quality=quality):
        ShowImage(data[0,:,:])    

def ExtractXSlice(offset,num_refinements):
    ExtractData([offset,offset+1],[0,dims[1]],[0,dims[2]],num_refinements)
    
def ExtractYSlice(offset,num_refinements):
    ExtractData([0,dims[0]],[offset,offset+1],[0,dims[2]],num_refinements)    
        

def ExtractZSlice(offset,num_refinements):
    ExtractData([0,dims[0]],[0,dims[1]],[offset,offset+1],num_refinements)

ipywidgets.interact(
    ExtractZSlice,
    offset=ipywidgets.widgets.IntSlider(value=int(dims[2]/2),min=0,max=dims[2]-1,step=1),
    num_refinements=ipywidgets.widgets.IntSlider(value=3,min=1,max=6,step=1))

interactive(children=(IntSlider(value=1024, description='offset', max=2047), IntSlider(value=3, description='n…

<function __main__.ExtractZSlice(offset, num_refinements)>